# Overall Info

In [64]:
%reload_kedro

[05/21/23 17:07:54] INFO     Resolved project path as:                                              ]8;id=189842;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=41241;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#132\132]8;;\
                             /home/work/Documents/repo/economic_freedom_ml.                                        
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/21/23 17:07:55] INFO     Kedro project economic_freedom_ml                                      ]8;id=152291;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=478392;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#101\101]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=382454;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=204814;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/ipython/__init__.py#102\102]8;;\
                             'pipelines'                                                                           

## Packages

In [2]:
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from prophet import Prophet
import xgboost as xgb
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from pandas_profiling import ProfileReport
from time import time

[05/20/23 18:37:00] WARNING  /home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site ]8;id=629393;file:///usr/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=415528;file:///usr/lib/python3.8/warnings.py#109\109]8;;\
                             -packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version                  
                             classes are deprecated. Use packaging.version instead.                                
                               if LooseVersion(mpl.__version__) >= "3.0":                                          
                                                                                                                   

                    WARNING  /home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site ]8;id=805925;file:///usr/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=366948;file:///usr/lib/python3.8/warnings.py#109\109]8;;\
                             -packages/setuptools/_distutils/version.py:345: DeprecationWarning:                   
                             distutils Version classes are deprecated. Use packaging.version                       
                             instead.                                                                              
                               other = LooseVersion(other)                                                         
                                                                                                                   

                    ERROR    Importing plotly failed. Interactive plots will not work.                   ]8;id=34428;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/plot.py\plot.py]8;;\:]8;id=536997;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/plot.py#39\39]8;;\

# Random Forest and XGBoost

## Checking Info

In [3]:
model_input = catalog.load("data_with_features")

[05/20/23 18:37:01] INFO     Loading data from 'data_with_features' (CSVDataSet)...             ]8;id=405306;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=215527;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [4]:
model_input.head()

,year,score,argentina,bahamas,barbados,belize,bolivia,brazil,canada,chile,...,peru,saint_lucia,saint_vincent_and_the_grenadines,suriname,trinidad_and_tobago,united_states,uruguay,venezuela,north_america,south_america
0,2023,50.1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2023,71.3,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2023,56.6,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2023,43.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2023,53.3,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
model_input_info = pd.DataFrame({'column': model_input.columns,  
                             'dtype': model_input.dtypes,
                             'missing_percent': model_input.isna().sum()/model_input.shape[0],
                             'count': model_input.count(),
                             'n_unique': model_input.nunique()}).reset_index().drop('index', axis=1).round(decimals = 3),  
print(model_input_info)

(                              column    dtype  missing_percent  count  \
0                               year    int64              0.0    883   
1                              score  float64              0.0    883   
2                          argentina    int64              0.0    883   
3                            bahamas    int64              0.0    883   
4                           barbados    int64              0.0    883   
5                             belize    int64              0.0    883   
6                            bolivia    int64              0.0    883   
7                             brazil    int64              0.0    883   
8                             canada    int64              0.0    883   
9                              chile    int64              0.0    883   
10                          colombia    int64              0.0    883   
11                        costa_rica    int64              0.0    883   
12                              cuba    int64     

## Preparing the Data

### Separating the data into feature data and target data (X_all and y_all, respectively)

In [6]:
model_input_ds = model_input.set_index("year")

In [7]:
X_all = model_input_ds.drop('score', axis=1)
y_all = model_input_ds['score']

In [8]:
X_all.head()

,argentina,bahamas,barbados,belize,bolivia,brazil,canada,chile,colombia,costa_rica,...,peru,saint_lucia,saint_vincent_and_the_grenadines,suriname,trinidad_and_tobago,united_states,uruguay,venezuela,north_america,south_america
year,,,,,,,,,,,,,,,,,,,,,
2023,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2023,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2023,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2023,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2023,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
y_all.head()

year
2023    50.1
2023    71.3
2023    56.6
2023    43.0
2023    53.3
Name: score, dtype: float64

### Training and Testing Data Split

In [10]:
X_train = X_all[X_all.index<2019]
y_train = y_all[y_all.index<2019]

X_test = X_all[X_all.index>=2019]
y_test = y_all[y_all.index>=2019]

print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 723 samples.
Testing set has 160 samples.


## Random Forest 

In [11]:
random_forest = RandomForestRegressor(random_state=999)
random_forest.fit(X_train, y_train)
rf_predicted = random_forest.predict(X_test)

## XGBoost

In [12]:
# fit the model
my_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000)
my_xgb.fit(X_train, y_train)

# predict on the same period
xb_predicted = my_xgb.predict(X_test)

In [13]:
xb_predicted

array([57.100132, 67.939   , 61.6377  , 56.558464, 57.045918, 75.73734 ,
       76.587364, 65.49149 , 66.53318 , 30.00015 , 64.110245, 58.575172,
       53.612682, 69.44567 , 62.0377  , 52.875107, 48.7918  , 57.775154,
       66.191505, 63.962643, 57.983517, 67.179016, 61.328976, 65.66232 ,
       69.529724, 66.79971 , 48.87844 , 70.98734 , 67.7781  , 77.52901 ,
       68.27484 , 44.71268 , 57.100132, 67.939   , 61.6377  , 56.558464,
       57.045918, 75.73734 , 76.587364, 65.49149 , 66.53318 , 30.00015 ,
       64.110245, 58.575172, 53.612682, 69.44567 , 62.0377  , 52.875107,
       48.7918  , 57.775154, 66.191505, 63.962643, 57.983517, 67.179016,
       61.328976, 65.66232 , 69.529724, 66.79971 , 48.87844 , 70.98734 ,
       67.7781  , 77.52901 , 68.27484 , 44.71268 , 57.100132, 67.939   ,
       61.6377  , 56.558464, 57.045918, 75.73734 , 76.587364, 65.49149 ,
       66.53318 , 30.00015 , 64.110245, 58.575172, 53.612682, 69.44567 ,
       62.0377  , 52.875107, 48.7918  , 57.775154, 66.191505, 63.962643,
       57.983517, 67.179016, 61.328976, 65.66232 , 69.529724, 66.79971 ,
       48.87844 , 70.98734 , 67.7781  , 77.52901 , 68.27484 , 44.71268 ,
       57.100132, 67.939   , 61.6377  , 56.558464, 57.045918, 75.73734 ,
       76.587364, 65.49149 , 66.53318 , 30.00015 , 64.110245, 58.575172,
       53.612682, 69.44567 , 62.0377  , 52.875107, 48.7918  , 57.775154,
       66.191505, 63.962643, 57.983517, 67.179016, 61.328976, 65.66232 ,
       69.529724, 66.79971 , 48.87844 , 70.98734 , 67.7781  , 77.52901 ,
       68.27484 , 44.71268 , 57.100132, 67.939   , 61.6377  , 56.558464,
       57.045918, 75.73734 , 76.587364, 65.49149 , 66.53318 , 30.00015 ,
       64.110245, 58.575172, 53.612682, 69.44567 , 62.0377  , 52.875107,
       48.7918  , 57.775154, 66.191505, 63.962643, 57.983517, 67.179016,
       61.328976, 65.66232 , 69.529724, 66.79971 , 48.87844 , 70.98734 ,
       67.7781  , 77.52901 , 68.27484 , 44.71268 ], dtype=float32)

# Prophet

## Checking Info

In [14]:
model_input_prophet = catalog.load("preprocess_data")

[05/20/23 18:37:02] INFO     Loading data from 'preprocess_data' (CSVDataSet)...                ]8;id=401484;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=375193;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [15]:
model_input_prophet.head()

,country,year,score,property_right,government_integrity,tax_burden,government_spending,business_freedom,labor_freedom,monetary_freedom,trade_freedom,investment_freedom,financial_freedom
0,argentina,2023,50.1,35.1,45.1,73.3,53.0,55.1,51.0,37.9,60.6,55.0,60.0
1,barbados,2023,71.3,72.6,68.7,80.6,70.8,64.7,63.4,78.6,58.4,70.0,60.0
2,belize,2023,56.6,34.7,38.1,77.0,64.1,54.0,59.9,82.5,55.6,55.0,50.0
3,bolivia,2023,43.0,14.1,28.7,86.0,58.8,54.7,46.2,72.8,60.8,15.0,40.0
4,brazil,2023,53.3,50.3,40.0,69.9,53.8,63.2,55.9,78.4,60.0,60.0,50.0


In [16]:
model_input_info = pd.DataFrame({'column': model_input_prophet.columns,  
                             'dtype': model_input_prophet.dtypes,
                             'missing_percent': model_input_prophet.isna().sum()/model_input_prophet.shape[0],
                             'count': model_input_prophet.count(),
                             'n_unique': model_input_prophet.nunique()}).reset_index().drop('index', axis=1).round(decimals = 3),  
print(model_input_info)

(                  column    dtype  missing_percent  count  n_unique
0                country   object              0.0    883        32
1                   year    int64              0.0    883        29
2                  score  float64              0.0    883       344
3         property_right  float64              0.0    883       165
4   government_integrity  float64              0.0    883       229
5             tax_burden  float64              0.0    883       309
6    government_spending  float64              0.0    883       390
7       business_freedom  float64              0.0    883       325
8          labor_freedom  float64              0.0    883       354
9       monetary_freedom  float64              0.0    883       332
10         trade_freedom  float64              0.0    883       246
11    investment_freedom  float64              0.0    883        19
12     financial_freedom  float64              0.0    883         9,)


### Separating the data into feature data and target data 
### (X_all and y_all, respectively)

In [17]:
model_input_prophet["year"] = (model_input_prophet["year"].map(str)+ "-12-31")

In [18]:
model_input_prophet["year"] = pd.to_datetime(model_input_prophet["year"])


In [19]:
model_input_prophet.head()

,country,year,score,property_right,government_integrity,tax_burden,government_spending,business_freedom,labor_freedom,monetary_freedom,trade_freedom,investment_freedom,financial_freedom
0,argentina,2023-12-31,50.1,35.1,45.1,73.3,53.0,55.1,51.0,37.9,60.6,55.0,60.0
1,barbados,2023-12-31,71.3,72.6,68.7,80.6,70.8,64.7,63.4,78.6,58.4,70.0,60.0
2,belize,2023-12-31,56.6,34.7,38.1,77.0,64.1,54.0,59.9,82.5,55.6,55.0,50.0
3,bolivia,2023-12-31,43.0,14.1,28.7,86.0,58.8,54.7,46.2,72.8,60.8,15.0,40.0
4,brazil,2023-12-31,53.3,50.3,40.0,69.9,53.8,63.2,55.9,78.4,60.0,60.0,50.0


In [20]:
y_all_prophet = model_input_prophet[['year','score','country']]
y_all_prophet = y_all_prophet.rename({'year': 'ds', 'score':'y'},axis=1)

In [21]:
y_all_prophet.head()

,ds,y,country
0,2023-12-31,50.1,argentina
1,2023-12-31,71.3,barbados
2,2023-12-31,56.6,belize
3,2023-12-31,43.0,bolivia
4,2023-12-31,53.3,brazil


### Training and Testing Data Split

In [22]:
y_train_prophet = y_all_prophet[y_all_prophet["ds"]<datetime(2019,1,1)]

y_test_prophet = y_all_prophet[y_all_prophet["ds"]>=datetime(2019,1,1)]

print("Training set has {} samples.".format(y_train_prophet.shape[0]))
print("Testing set has {} samples.".format(y_test_prophet.shape[0]))

Training set has 723 samples.
Testing set has 160 samples.


## Prophet (Meta) Predictions

In [23]:
# Assuming your dataset is in a DataFrame called 'data', with columns 'ds', 'y', and 'country'
countries = y_train_prophet['country'].unique()
predictions = []

for country in countries:
    # Filter data for the current country
   country_data = y_train_prophet[y_train_prophet['country'] == country]
   
   # Define and fit the model
   model = Prophet()
   model.fit(country_data)
   
   # Create a DataFrame for future dates
   future = model.make_future_dataframe(periods=5, freq='Y') # 5 years of daily predictions
   
   # Make predictions and store them in the 'predictions' list
   forecast = model.predict(future)
   forecast['country'] = country
   predictions.append(forecast)

# Combine all predictions into a single DataFrame
all_predictions = pd.concat(predictions, ignore_index=True)

                    INFO     Disabling weekly seasonality. Run prophet with                       ]8;id=49505;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/forecaster.py\forecaster.py]8;;\:]8;id=990909;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/forecaster.py#931\931]8;;\
                             weekly_seasonality=True to override this.                                             

                    INFO     Disabling daily seasonality. Run prophet with daily_seasonality=True ]8;id=881090;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/forecaster.py\forecaster.py]8;;\:]8;id=813296;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/forecaster.py#931\931]8;;\
                             to override this.                                                                     

                    INFO     n_changepoints greater than number of observations. Using 18.        ]8;id=420038;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/forecaster.py\forecaster.py]8;;\:]8;id=72416;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/prophet/forecaster.py#401\401]8;;\

                    DEBUG    input tempfile: /tmp/tmpsaail0o2/ayt14qm8.json                       ]8;id=803906;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/utils/filesystem.py\filesystem.py]8;;\:]8;id=654836;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/utils/filesystem.py#123\123]8;;\

                    DEBUG    input tempfile: /tmp/tmpsaail0o2/r7q2rw3g.json                       ]8;id=232529;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/utils/filesystem.py\filesystem.py]8;;\:]8;id=704285;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/utils/filesystem.py#123\123]8;;\

...

                    DEBUG    CmdStan args:                                                            ]8;id=191680;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/model.py\model.py]8;;\:]8;id=267856;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/model.py#1690\1690]8;;\
                             ['/home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site              
                             -packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=50407',              
                             'data', 'file=/tmp/tmpsaail0o2/psp0ldne.json',                                        
                             'init=/tmp/tmpsaail0o2/v5u9_ik9.json', 'output',                                      
                             'file=/tmp/tmpsaail0o2/prophet_modelo_dnl2rz/prophet_model-2023052018371              
                             1.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']                          

18:37:11 - cmdstanpy - INFO - Chain [1] start processing


                    INFO     Chain [1] start processing                                               ]8;id=605166;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/model.py\model.py]8;;\:]8;id=558246;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/model.py#1693\1693]8;;\

18:37:11 - cmdstanpy - INFO - Chain [1] done processing


                    INFO     Chain [1] done processing                                                ]8;id=983624;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/model.py\model.py]8;;\:]8;id=801050;file:///home/work/Documents/repo/economic_freedom_ml/.venv/lib/python3.8/site-packages/cmdstanpy/model.py#1750\1750]8;;\

In [24]:
all_predictions[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(15)

,ds,yhat,yhat_lower,yhat_upper
868,2009-12-31,40.551589,37.425063,43.751471
869,2010-12-31,39.638063,36.192546,43.097936
870,2011-12-31,39.884251,36.496038,43.214422
871,2012-12-31,37.215452,33.892822,40.342610
872,2013-12-31,35.141647,31.881878,38.403215
873,2014-12-31,34.228121,30.902177,37.487259
874,2015-12-31,34.474309,31.185383,37.728783
875,2016-12-31,31.805510,28.347669,35.322650
876,2017-12-31,29.731705,26.389024,33.030129
877,2018-12-31,28.818179,25.446187,31.972671


In [35]:
all_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ds                          883 non-null    datetime64[ns]
 1   trend                       883 non-null    float64       
 2   yhat_lower                  883 non-null    float64       
 3   yhat_upper                  883 non-null    float64       
 4   trend_lower                 883 non-null    float64       
 5   trend_upper                 883 non-null    float64       
 6   additive_terms              883 non-null    float64       
 7   additive_terms_lower        883 non-null    float64       
 8   additive_terms_upper        883 non-null    float64       
 9   yearly                      883 non-null    float64       
 10  yearly_lower                883 non-null    float64       
 11  yearly_upper                883 non-null    float64       

In [25]:
for_testing = all_predictions[all_predictions["ds"]>=datetime(2019,1,1)]

arranging_order_by_merge = pd.merge(
    y_test_prophet, 
    for_testing,
    how="left",
    on=['country', 'ds'], 
)
arranging_order_by_merge

,ds,y,country,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-12-31,50.1,argentina,-70.928620,29.444645,39.794940,-70.934087,-70.922284,105.700535,105.700535,105.700535,105.700535,105.700535,105.700535,0.0,0.0,0.0,34.771914
1,2023-12-31,71.3,barbados,16.049490,59.955114,70.593762,16.039950,16.059364,49.271991,49.271991,49.271991,49.271991,49.271991,49.271991,0.0,0.0,0.0,65.321481
2,2023-12-31,56.6,belize,-14.424758,51.392027,55.899910,-14.764212,-14.038202,68.091918,68.091918,68.091918,68.091918,68.091918,68.091918,0.0,0.0,0.0,53.667160
3,2023-12-31,43.0,bolivia,-57.629360,33.706406,42.156719,-57.634039,-57.624445,95.634875,95.634875,95.634875,95.634875,95.634875,95.634875,0.0,0.0,0.0,38.005516
4,2023-12-31,53.3,brazil,7.522393,51.124498,61.591674,7.515034,7.530327,49.260761,49.260761,49.260761,49.260761,49.260761,49.260761,0.0,0.0,0.0,56.783155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2019-12-31,62.9,bahamas,6.888436,62.844308,67.544824,6.878469,6.899163,58.318021,58.318021,58.318021,58.318021,58.318021,58.318021,0.0,0.0,0.0,65.206458
156,2019-12-31,57.0,trinidad_and_tobago,23.038521,57.523684,60.856192,23.009711,23.071937,36.134598,36.134598,36.134598,36.134598,36.134598,36.134598,0.0,0.0,0.0,59.173119
157,2019-12-31,76.8,united_states,-6.087406,73.625790,75.388299,-6.156137,-5.995586,80.613719,80.613719,80.613719,80.613719,80.613719,80.613719,0.0,0.0,0.0,74.526313
158,2019-12-31,68.6,uruguay,-11.997996,67.759999,71.080105,-12.015566,-11.976551,81.462023,81.462023,81.462023,81.462023,81.462023,81.462023,0.0,0.0,0.0,69.464027


# R2 Score

## Random Forest

In [26]:
print("R2",r2_score(y_test,rf_predicted))

R2 0.7517776244569526


In [27]:
print("MAE",mean_absolute_error(y_test,rf_predicted))

MAE 3.8846376048263225


In [28]:
print("MSE",mean_squared_error(y_test,rf_predicted))

MSE 33.39780469523098


## XG Boost

In [29]:
print("R2",r2_score(y_test,xb_predicted))

R2 0.7549268081277446


In [30]:
print("MAE",mean_absolute_error(y_test,xb_predicted))

MAE 3.8705165696144106


In [31]:
print("MSE",mean_squared_error(y_test,xb_predicted))

MSE 32.97408857795339


## Prophet

In [32]:
print("R2",r2_score(arranging_order_by_merge['y'],arranging_order_by_merge["yhat"]))

R2 0.8504568685329247


In [33]:
print("MAE",mean_absolute_error(arranging_order_by_merge['y'],arranging_order_by_merge["yhat"]))

MAE 2.868168091042787


In [34]:
print("MSE",mean_squared_error(arranging_order_by_merge['y'],arranging_order_by_merge["yhat"]))

MSE 20.12071751115962
